In [1]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir ('/content/drive/My Drive')
print(os.listdir('/content/drive/My Drive/datasets'))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/
['testhash.csv', 'trainhash.csv']


In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np

import csv

h_train = []
l_train = []
path = '/content/drive/My Drive/datasets/'
print('Started!')
print('Data loading started\/\/\/\/\/\/\/\/\/\/')
i=0
with open(path+'trainhash.csv') as myFile:
    reader = csv.DictReader(myFile)
    for row in reader:
        # if(i==5):
        #     break
        # print(row['hash'])
        # i += 1
        re = str(bin(int(row['hash'], 16)))[2:]

        while(len(re) != 256):
            re = '0'+ re

        h_train.append(np.array(list(re)))
        # l_train.append(row['len'])
# print(h_train[0].shape)
# for i in range(0,5):
#     print(h_train[i].shape)
        datalen = int(row['len'])-1
        l_train.append(datalen)
        # if(num_classes < datalen):
        #     num_classes = datalen


h_train = np.array(h_train)
l_train = np.array(l_train)
print('Training data loaded.\nTraining size is {}'.format(h_train[0].shape[0]))

h_test = []
l_test = []

num_classes = 0

with open(path+'testhash.csv') as myFile:
    reader = csv.DictReader(myFile)
    for row in reader:
        # print(row['hash'])
        re = str(bin(int(row['hash'], 16)))[2:]

        while (len(re) != 256):
            re = '0' + re

        h_test.append(np.array(list(re)))

        # h_test.append(list(str(bin(int(row['hash'], 16)))[2:]))
        datalen = int(row['len'])-1
        l_test.append(datalen)
        if(num_classes < datalen):
            num_classes = datalen

print('Classes: '+str(num_classes))
print(l_test[250])
h_test = np.array(h_test)
l_test = np.array(l_test)

print('Testind data loaded.\nTesting size is {}'.format(h_test.shape))
print('Data is loaded!!\n Training is started!')

batch_size = 128

epochs = 12

# input image dimensions
img_rows, img_cols = 16, 16

# the data, split between train and test sets
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = h_train.reshape(h_train.shape[0],16,16,1)
x_test = h_test.reshape(h_test.shape[0],16,16,1)


print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train = l_train
y_test = l_test

num_classes = num_classes + 1

# Y = 2 # the value 2 represents that the hash is of input size digit 2
#
# Y = [0,0,1,0] # The 2nd position in the vector is made 1
#
# # Here, the class value is converted into a binary class matrix

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(16,16,1)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.summary()

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', 100*score[1],'%')


Started!
Data loading started\/\/\/\/\/\/\/\/\/\/
Training data loaded.
Training size is 256
Classes: 3
3
Testind data loaded.
Testing size is (2500, 256)
Data is loaded!!
 Training is started!
x_train shape: (7500, 16, 16, 1)
7500 train samples
2500 test samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2304)              0         
__________